# 個別詳細分析

特定銘柄の詳細分析を行います。


In [19]:
import sys
import os
from pathlib import Path

# プロジェクトルートをパスに追加
# Jupyter Notebookのカレントディレクトリからプロジェクトルートを取得
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    # notebooks/ディレクトリから実行している場合
    project_root = current_dir.parent
else:
    # プロジェクトルートから実行している場合
    project_root = current_dir

sys.path.insert(0, str(project_root))

# デバッグ用（必要に応じてコメントアウト）
# print(f"Current directory: {current_dir}")
# print(f"Project root: {project_root}")
# print(f"Python path: {sys.path[:3]}")

from src.api import JQuantsAPIClient
from src.analysis import IndividualAnalyzer
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [20]:
# キャッシュをクリアする場合（古いデータが残っている場合）
# 必要に応じてこのセルを実行してください
from src.utils.cache import CacheManager
cache = CacheManager()
cache.clear()  # 全キャッシュをクリア
print("✅ キャッシュをクリアしました")


✅ キャッシュをクリアしました


## 1. 分析対象銘柄の設定


In [21]:
# 分析対象銘柄コードの設定
# 環境変数NOTEBOOK_STOCK_CODESが設定されている場合はそれを使用
# 設定されていない場合は入力を求める
if "NOTEBOOK_STOCK_CODES" in os.environ:
    stock_codes_str = os.environ.get("NOTEBOOK_STOCK_CODES", "")
    stock_codes = [code.strip() for code in stock_codes_str.split(",") if code.strip()]
    print(f"環境変数から銘柄コードを読み込みました: {stock_codes}")
else:
    # ユーザーに入力を求める
    print("分析対象銘柄コードを入力してください")
    print("例: 6501 または 6501,2802 (複数銘柄の場合はカンマ区切り)")
    stock_codes_input = input("銘柄コード: ")
    stock_codes = [code.strip() for code in stock_codes_input.split(",") if code.strip()]
    if not stock_codes:
        raise ValueError("銘柄コードが入力されていません")
    print(f"分析対象銘柄: {stock_codes}")
# アナライザーを初期化
api_client = JQuantsAPIClient()
analyzer = IndividualAnalyzer(api_client, use_cache=False)  # キャッシュを無効化（最新データを取得）


分析対象銘柄コードを入力してください
例: 6501 または 6501,2802 (複数銘柄の場合はカンマ区切り)
分析対象銘柄: ['6506']


## 2. データ取得と分析


In [22]:
results = []

for code in stock_codes:
    print(f"分析中: {code}")
    result = analyzer.get_report_data(code)
    # デバッグ: 取得されたデータの年数を確認
    if result:
        name = result.get("name", "")
        name_display = f" {name}" if name else ""
        metrics = result.get("metrics", {})
        years = metrics.get("years", [])
        analysis_years = metrics.get("analysis_years", 0)
        print(f"  ✅ 取得完了: {len(years)}年分のデータ（分析年数: {analysis_years}年）{name_display}")
    if result:
        results.append(result)
    else:
        print(f"  ⚠️  {code} の分析に失敗しました")

print(f"\n分析完了: {len(results)}銘柄")



分析中: 6506
🔍 get_report_data: 6506 安川電機 の分析を開始します（キャッシュ: 無効）
📥 APIから取得された財務データ: 20件 安川電機
📥 年度データ（CurPerType='FY'）: 5件 安川電機
  年度終了日一覧:
    2021-02-28 (開示日: 2021-04-09)
    2022-02-28 (開示日: 2022-04-08)
    2023-02-28 (開示日: 2023-04-07)
    2024-02-29 (開示日: 2024-04-05)
    2025-02-28 (開示日: 2025-04-04)
📊 取得された年度データ: 5年分 安川電機
  1. 年度終了日: 2025-02-28, 開示日: 2025-04-04
  2. 年度終了日: 2024-02-29, 開示日: 2024-04-05
  3. 年度終了日: 2023-02-28, 開示日: 2023-04-07
  4. 年度終了日: 2022-02-28, 開示日: 2022-04-08
  5. 年度終了日: 2021-02-28, 開示日: 2021-04-09
📈 分析年数: 5年（利用可能: 5年、最大: 10年） 安川電機
🔧 calculate_metrics_flexible: 分析年数=5, 利用可能年数=5
🔧 calculate_metrics_flexible: 実際に使用する年数=5
✅ 分析完了: 5年分のデータ（分析年数: 5年） 安川電機
  ✅ 取得完了: 5年分のデータ（分析年数: 5年） 安川電機

分析完了: 1銘柄


## 3. ビジュアルHTMLレポート生成（プレゼンテーション品質）

分析結果を美しいHTMLレポートとして出力します。


In [23]:
# モジュールを再読み込み（コード更新時）
import importlib
import sys

# 依存モジュールを先に再読み込み
if "src.analysis.individual" in sys.modules:
    importlib.reload(sys.modules["src.analysis.individual"])
if "src.analysis" in sys.modules:
    importlib.reload(sys.modules["src.analysis"])

# HTMLReportGeneratorを再読み込み
if "src.report.html_report" in sys.modules:
    importlib.reload(sys.modules["src.report.html_report"])
if "src.report" in sys.modules:
    importlib.reload(sys.modules["src.report"])

print("✅ モジュールを再読み込みしました")


✅ モジュールを再読み込みしました


In [26]:

# ビジュアルHTMLレポート生成
from datetime import datetime
from pathlib import Path

from src.report import HTMLReportGenerator

# 出力ディレクトリを作成
output_dir = project_root / "reports"
output_dir.mkdir(exist_ok=True)

print("=" * 60)
print("ビジュアルHTMLレポート生成")
print("=" * 60)
print()

for result in results:
    code = result.get("code")
    name = result.get("name")
    
    if not result.get("metrics", {}).get("years"):
        print(f"⚠️  {code}: データがありません")
        continue
    
    # ファイル名を生成
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    html_filename = f"visual_report_{code}_{timestamp}.html"
    html_path = output_dir / html_filename
    
    print(f"生成中: {code} {name}")
    print(f"出力先: {html_path}")
    print()
    
    try:
        # HTMLレポート生成器を初期化
        generator = HTMLReportGenerator()
        
        # レポート生成
        success = generator.generate(
            result=result,
            output_path=str(html_path)
        )
        
        if success:
            print(f"✅ HTMLレポートが生成されました")
            print(f"   ファイル: {html_path}")
            print(f"   ファイルサイズ: {html_path.stat().st_size / 1024:.1f} KB")
            print()
            print("【ブラウザで開く方法】")
            print(f"   open {html_path}  # macOS")
        
        print()
        
    except ImportError as e:
        print(f"❌ エラー: 必要なパッケージがインストールされていません")
        print(f"   {e}")
        print()
        print("【インストール方法】")
        print("   pip install jinja2 plotly")
        print()
    except Exception as e:
        print(f"❌ エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
        print()

print(f"出力ディレクトリ: {output_dir}")


ビジュアルHTMLレポート生成

生成中: 6506 安川電機
出力先: /Users/sollahiro/coding/jquants-analyzer/reports/visual_report_6506_20260104_194658.html

✅ HTMLレポートが生成されました: /Users/sollahiro/coding/jquants-analyzer/reports/visual_report_6506_20260104_194658.html

✅ HTMLレポートが生成されました
   ファイル: /Users/sollahiro/coding/jquants-analyzer/reports/visual_report_6506_20260104_194658.html
   ファイルサイズ: 82.8 KB

【ブラウザで開く方法】
   open /Users/sollahiro/coding/jquants-analyzer/reports/visual_report_6506_20260104_194658.html  # macOS

出力ディレクトリ: /Users/sollahiro/coding/jquants-analyzer/reports
